# Computational Theory Assessment: Winter 25/26

In [ ]:
import numpy as np
import math
import struct
from typing import Generator

## Problem 1: Binary Words and Operations

In [ ]:
# Main SHA-256 functions
def Parity(x, y, z):
    """
    Calculate the parity of three 32-bit words.
    
    The parity function returns 1 when an odd number of inputs are 1, 
    and 0 when an even number of inputs are 1. Equivalent to x XOR y XOR z.
    
    Parameters:
        x (int): First 32-bit integer
        y (int): Second 32-bit integer  
        z (int): Third 32-bit integer
        
    Returns:
        numpy.uint32: x XOR y XOR z
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return x ^ y ^ z

def Ch(x, y, z):
    """
    Choose between y and z based on value of x.
    
    For each bit position, chooses y if x is 1, z if x is 0.
    Equivalent to (x AND y) XOR (NOT x AND z).
    
    Parameters:
        x (int): 32-bit integer used as selector
        y (int): 32-bit integer chosen when x=1
        z (int): 32-bit integer chosen when x=0
        
    Returns:
        numpy.uint32: (x AND y) XOR (NOT x AND z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    """
    Calculate majority value at each bit position.
    
    For each bit position, returns 1 if at least two inputs are 1.
    Equivalent to (x AND y) XOR (x AND z) XOR (y AND z).
    
    Parameters:
        x (int): First 32-bit integer
        y (int): Second 32-bit integer
        z (int): Third 32-bit integer
        
    Returns:
        numpy.uint32: Majority value at each bit position
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (x & z) ^ (y & z)

In [ ]:
def Sigma0(x):
    """
    SHA-256 uppercase Sigma0 function.
    
    Performs three bitwise rotations and XORs the results.
    Equivalent to ROTR-2(x) XOR ROTR-13(x) XOR ROTR-22(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three rotations XORed together
    """
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_2 = (x >> 2) | (x << (32 - 2))
    rotr_13 = (x >> 13) | (x << (32 - 13))
    rotr_22 = (x >> 22) | (x << (32 - 22))
    return rotr_2 ^ rotr_13 ^ rotr_22

def Sigma1(x):
    """
    SHA-256 uppercase Sigma1 function.
    
    Performs three bitwise rotations and XORs the results.
    Equivalent to ROTR-6(x) XOR ROTR-11(x) XOR ROTR-25(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three rotations XORed together
    """
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_6 = (x >> 6) | (x << (32 - 6))
    rotr_11 = (x >> 11) | (x << (32 - 11))
    rotr_25 = (x >> 25) | (x << (32 - 25))
    return rotr_6 ^ rotr_11 ^ rotr_25

def sigma0(x):
    """
    SHA-256 lowercase sigma0 function.
    
    Performs two rotations and one shift, then XORs the results.
    Equivalent to ROTR-7(x) XOR ROTR-18(x) XOR SHR-3(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three operations XORed together
    """
    x = np.uint32(x)
    # Inline implementation
    rotr_7 = (x >> 7) | (x << (32 - 7))
    rotr_18 = (x >> 18) | (x << (32 - 18))
    shr_3 = x >> 3
    return rotr_7 ^ rotr_18 ^ shr_3

def sigma1(x):
    """
    SHA-256 lowercase sigma1 function.
    
    Performs two rotations and one shift, then XORs the results.
    Equivalent to ROTR-17(x) XOR ROTR-19(x) XOR SHR-10(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three operations XORed together
    """
    x = np.uint32(x)
    # Inline implementation
    rotr_17 = (x >> 17) | (x << (32 - 17))
    rotr_19 = (x >> 19) | (x << (32 - 19))
    shr_10 = x >> 10
    return rotr_17 ^ rotr_19 ^ shr_10

In [ ]:
"""
## Testing and Verification

We test all 7 required functions with representative 32-bit values 
to verify correctness according to the SHA-256 specification.
"""

print("=== Testing Required SHA-256 Functions ===\n")

print("1. Testing Parity, Ch, and Maj functions:")
print(f"Parity(0x00000001, 0x00000001, 0x00000000) = {Parity(0x00000001, 0x00000001, 0x00000000):08x}")
print(f"Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = {Ch(0xFFFFFFFF, 0x12345678, 0x87654321):08x}")
print(f"Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = {Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F):08x}")
print()

print("2. Testing Sigma functions:")
test_val = 0x12345678
print(f"Sigma0(0x12345678) = {Sigma0(test_val):08x}")
print(f"Sigma1(0x12345678) = {Sigma1(test_val):08x}")
print(f"sigma0(0x12345678) = {sigma0(test_val):08x}")
print(f"sigma1(0x12345678) = {sigma1(test_val):08x}")

## Problem 2: Fractional Parts of Cube Roots

In [ ]:
def primes(n):
    """
    Generate the first n prime numbers using the Sieve of Eratosthenes.
    
    Parameters:
        n (int): Number of prime numbers to generate
        
    Returns:
        list: First n prime numbers in ascending order
        
    Raises:
        ValueError: If n is not a positive integer
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("n must be a positive integer")
    
    if n == 1:
        return [2]
    
    # Estimate upper bound for the nth prime
    if n < 6:
        upper_bound = 20
    else:
        upper_bound = int(n * (math.log(n) + math.log(math.log(n)))) + 10
    
    sieve = [True] * (upper_bound + 1)
    sieve[0:2] = [False, False]
    
    primes_list = []
    for num in range(2, upper_bound + 1):
        if sieve[num]:
            primes_list.append(num)
            if len(primes_list) == n:
                break
            for multiple in range(num * num, upper_bound + 1, num):
                sieve[multiple] = False
    
    return primes_list

# Test prime number generation
print("=== Testing Prime Number Generator ===\n")

test_cases = [1, 5, 10, 64]
for n in test_cases:
    prime_list = primes(n)
    print(f"First {n} primes: {prime_list}")
    print(f"Length: {len(prime_list)}, Last prime: {prime_list[-1]}\n")

In [ ]:
def get_fractional_bits_corrected(number, num_bits=32):
    """
    Extract the first num_bits of fractional part using high-precision method.
    
    This implementation follows the exact specification: take the fractional part,
    multiply by 2^num_bits, and take the integer part (floor).
    
    Parameters:
        number (float): Input number
        num_bits (int): Number of bits to extract (default 32)
        
    Returns:
        int: Integer representing the first num_bits of fractional part
    """
    # Get fractional part (number modulo 1)
    fractional = number - int(number)
    
    # Multiply by 2^num_bits to shift bits to integer part
    # Use integer conversion to get floor value
    scaled = fractional * (2 ** num_bits)
    
    # Convert to integer (this takes the floor)
    result = int(scaled)
    
    # Ensure we only have num_bits
    return result & ((1 << num_bits) - 1)

# Test fractional part extraction
print("=== Testing Fractional Part Bit Extraction ===\n")

test_primes = [2, 3, 5, 7]
for prime in test_primes:
    cube_root = prime ** (1.0/3.0)
    fractional_bits = get_fractional_bits_corrected(cube_root, 32)
    print(f"Prime: {prime}")
    print(f"Cube root: {cube_root:.10f}")
    print(f"Fractional part bits: {fractional_bits:08x}")
    print(f"Integer part: {int(cube_root)}, Fractional: {cube_root - int(cube_root):.10f}\n")

In [ ]:
def calculate_sha256_constants_corrected():
    """
    Calculate SHA-256 constants with exact FIPS PUB 180-4 specification.
    
    Uses high-precision cube root calculation to match the standard exactly.
    
    Returns:
        list: List of 64 hexadecimal strings representing the constants
    """
    prime_numbers = primes(64)
    constants = []
    
    for prime in prime_numbers:
        # Calculate cube root with high precision
        cube_root = prime ** (1.0/3.0)
        
        # Extract first 32 bits of fractional part
        fractional_bits = get_fractional_bits_corrected(cube_root, 32)
        
        # Convert to hexadecimal (8 characters for 32 bits)
        hex_constant = f"{fractional_bits:08x}"
        constants.append(hex_constant)
    
    return constants

# Calculate all SHA-256 constants
print("=== SHA-256 Constants Calculation ===\n")

sha256_constants = calculate_sha256_constants_corrected()

print("Generated SHA-256 Constants:")
print("-" * 40)

# Display in standard format
def format_constants(constants, items_per_line=8):
    """Format constants for display in standard format."""
    lines = []
    for i in range(0, len(constants), items_per_line):
        line = " ".join(constants[i:i + items_per_line])
        lines.append(line)
    return "\n".join(lines)

print(format_constants(sha256_constants))
print(f"\nTotal constants generated: {len(sha256_constants)}")

In [ ]:
# Expected constants from FIPS PUB 180-4
expected_constants = [
    '428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5',
    'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174',
    'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da',
    '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967',
    '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85',
    'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070',
    '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3',
    '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2'
]

print("=== Verification Against FIPS PUB 180-4 ===\n")

print("Generated Constants vs Expected:")
print("-" * 50)

matches = 0
for i in range(64):
    gen = sha256_constants[i]
    exp = expected_constants[i]
    status = "MATCH" if gen == exp else "MISMATCH"
    if gen == exp:
        matches += 1
    print(f"{i:2d}. {gen} {status:8} {exp}")

print(f"\nMatch Results: {matches}/64 correct ({matches/64*100:.1f}%)")

if matches == 64:
    print("SUCCESS: All constants match the FIPS PUB 180-4 standard!")
else:
    print(f"There are {64-matches} constants that don't match.")
    print("\nFirst 5 mismatches:")
    mismatch_count = 0
    for i in range(64):
        if sha256_constants[i] != expected_constants[i]:
            print(f"  Constant {i}: Generated {sha256_constants[i]}, Expected {expected_constants[i]}")
            mismatch_count += 1
            if mismatch_count >= 5:
                break

## Problem 3: Padding

In [81]:
def block_parse(msg: bytes) -> Generator[bytes, None, None]:
    """
    Parse and pad a message into 512-bit blocks for SHA-256.
    
    This generator implements the SHA-256 padding standard:
    1. Append '1' bit (as 0x80 byte)
    2. Append k zero bits where ℓ + 1 + k ≡ 448 mod 512
    3. Append 64-bit message length in bits
    4. Yield 512-bit (64-byte) blocks
    
    Parameters:
        msg (bytes): Input message to pad and parse
        
    Yields:
        bytes: Next 512-bit block (64 bytes) of padded message
    """
    # Message length in bits
    msg_len_bits = len(msg) * 8
    
    # Start with message
    padded = bytearray(msg)
    
    # Step 1: Append '1' bit as 0x80 byte
    padded.append(0x80)
    
    # Step 2: Calculate and append zero bits
    current_bits = len(padded) * 8
    zero_bits = (448 - current_bits) % 512
    zero_bytes = (zero_bits + 7) // 8  # Ceiling division
    padded.extend(b'\x00' * zero_bytes)
    
    # Step 3: Append 64-bit message length
    padded.extend(struct.pack('>Q', msg_len_bits))
    
    # Step 4: Yield 512-bit blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

In [82]:
"""
## Testing the block_parse Generator

We test 5 different message lengths to verify correct padding.
"""


print("=== Testing block_parse() with 5 Different Message Lengths ===\n")

# Test 1: Empty message
print("Test 1: Empty message (0 bytes)")
blocks = list(block_parse(b''))
print(f"  Blocks: {len(blocks)}")
print(f"  First byte: 0x{blocks[0][0]:02x} (0x80 = '1' bit)")
print(f"  Last 8 bytes: {blocks[0][-8:].hex()} (length = 0)")
print()

# Test 2: "abc" 
print("Test 2: 'abc' (3 bytes, FIPS standard example)")
blocks = list(block_parse(b'abc'))
print(f"  Blocks: {len(blocks)}")
block_hex = blocks[0].hex()
print(f"  Block starts: {block_hex[:32]}...")
print(f"  Block ends: ...{block_hex[-16:]}")
print(f"  Length field: {struct.unpack('>Q', blocks[0][-8:])[0]} bits")
print()

=== Testing block_parse() with 5 Different Message Lengths ===

Test 1: Empty message (0 bytes)
  Blocks: 1
  First byte: 0x80 (0x80 = '1' bit)
  Last 8 bytes: 0000000000000000 (length = 0)

Test 2: 'abc' (3 bytes, FIPS standard example)
  Blocks: 1
  Block starts: 61626380000000000000000000000000...
  Block ends: ...0000000000000018
  Length field: 24 bits



# Problem 4: Hashes

# Problem 5: Passwords